# Math 134C Week 1

## Review of the options
A **call option** is a contract with a expiry date $T>t$ and a **strike price/exercise price** $K>0$ in which
1. The holder (who has the long position) has the **right** to buy the underlying asset at the strike price at the expiry date. (This is the European option. For American option, holder can choose to exercise the right anytime before the expiry date)
2. The writer (who has the short position) is obliged to deliver the underlying asset at the strike price if the holder choose to exercise the right.

The payoff of holding the call option at the expiry date is 
$$C_T(S_T, K) = (S_T-K)^+ = \max(S_T-K, 0)$$
The holder of the contract needs to pay a positive amount of money to the writer (this is a consequence of no arbitrage). We call that as **premium**. We will learn in this class about what is the right price for a call option.

A **put option** is defined similarly. The difference is that the holder has the right to sell the underlying asset at the strike price. The payoff for the holder is 
$$P_T(S_T, K) = (K-S_T)^+$$

Also, note that if a stock pays dividend, then the holder of the call option does not have the right to receive the dividend. Usually the amound of dividend paid will be the amount that the stock price decreases. 

## Put-Call Parity
Consider a call option and a put option written on the same underlying asset with price $S_t$, with the same strike price and also same expiry date $T$. Suppose the risk-free factor is $r$. Then the price of call option and put option $C_t, P_t$ satisfies
$$C_t-P_t = S_t-Ke^{-r(T-t)}$$

**Proof:**
We consider a portfolio consisting of longing a call option and short a put option. Then its payoff at time $T$ is $$(S_T-K)^+ -(K-S_T)^+ = S_T-K.$$ Consider another portfolio of long a share of stock and short $Ke^{-r(T-t)}$ in the bank, then the payoff at time $T$ is also $S_T-K$. Since those two portfolio have the same payoff at time $T$, their value at time $t$ must also be the same by the no arbitrage principle. Hence, we get $C_t-P_t = S_t-Ke^{-r(T-t)}$

In [25]:
import math
def put_given_call(C_t, S_t, K, r, dt):
    return C_t - S_t + K * math.exp(-r * dt)
def call_given_put(P_t, S_t, K, r, dt):
    return P_t + S_t - K * math.exp(-r * dt)

### Example in the lecture 1 note. 

Question: There are two days until expiration date. Yuting wants to sell a call option with strike price 100; i.e., she wants to go short on $C_100$. The interest rate is $r = 10\%$ and the current value of the stock is 120. Use the put-call parity expression to find a lower bound on the value of this Call.

Answer: The lower bound of the call option price is reached when the put option price is minimized. The minimal put option price is $0$. So we run the following code to find the lower bound for the call option price. 

In [26]:
call_given_put(P_t = 0, S_t = 120, K = 100, r = 0.1, dt = 2/365)

20.05477951109212

### Put-Call parity for dividend paying stock
Suppose the stock will pay a dividend of amount $D$ at a time between $t$ and $T$. Then the value of the stock at $t$ should be adjusted to $S_t - PV(D)$ given that we are not receiving the dividend. So we get the formula 
$$C_t-P_t = S_t-PV(D) - Ke^{-r(T-t)}$$
For continuous dividend paying stock with continuous dividend yield $\delta$, the future value of the stock with the dividend at the time $T$ is given by $S_Te^{\delta (T-t)}$. Hence it is enough to hold $e^{-\delta(T-t)}$ share of stock, and its payoff (future value) at time $T$ is $S_T$. Then we get the formula

$$C_t - P_t = S_te^{-\delta(T-t)} - Ke^{-r(T-t)}$$ 

In [27]:
import math
# r is the risk-free rate, F is a cash flow, and dT is the time that we get each cash flow
def PV(F, dT, r):
    pv = 0
    if len(dT) != len(F):
        print("Input Error")
        return 0
    for n in range(0, len(F)):
        pv += F[n] * math.exp(-r * dT[n])
    return pv
        
# C_t is the price of call option, S_t is the current stock price, K is the strike, r is the risk-free rate, dt is time to expiration. D is the dividend of the stock, and dT is the time at which we receive the dividend. The delta is for the continuous dividend paying model. The function will return the price of the corresponding put option. 
def put_given_call(C_t, S_t, K, r, dt, D= None, dT = None, delta = 0):
    if D is None: D = []
    if dT is None: dT = []
    return C_t - (S_t - PV(D, dT, r)) * math.exp(-delta * dt) + K * math.exp(-r * dt)

def call_given_put(P_t, S_t, K, r, dt, D = None, dT = None, delta = 0):
    if D is None: D = []
    if dT is None: dT = []
    return P_t + (S_t - PV(D, dT, r)) * math.exp(-delta * dt) - K * math.exp(-r * dt)

### Example
Problem 9.1 In textbook

A stock currently sells for 32.00. A 6-month call option with a strike of 35.00 has a premium of 2.27. Assuming a 0.04 continuously compounded risk-free rate and a 0.06 continuous dividend yield, what is the price of the associated put option?

In [28]:
put_given_call(C_t = 2.27, S_t = 32, K = 35, r = 0.04, dt = 0.5, delta = 0.06)

5.522696492184174

### Example
Jafee Corp. common stock is priced at 36.50 per share. The company just paid its 0.50 quarterly dividend. The continuously compounded risk-free annual interest rate is 0.06. A 35.00 strike European call, maturing in 6 months, sells for 3.20. What is the price of a 6-month, 35.00 strike put option?

In [29]:
put_given_call(C_t = 3.2, S_t = 36.5, K = 35, r = 0.06, D=[0.5, 0.5], dT = [0.25, 0.5], dt = 0.5)

1.6433724107735728

### Arbitrage opportunity for mispriced options 
Consider the option on a non-dividend paying stock for simplicity. If the put-call parity equation does not hold, then we will have arbitrage opportunities. The arbitrage profit we get is the difference between two sides of the inequality. If
$$C_t-P_t > S_t-Ke^{-r(T-t)}$$
Then we will long the cheaper side and short the expensive side. More precisely, we create a portfolio by purchasing a stock, a put option, and we sell a call option, and borrow $Ke^{-r(T-t)}$ from the bank. Then we currently have 
$$S_t-Ke^{-r(T-t)}+P_t-C_t>0$$
At expiry date, we have costless liquidation, and the arbitrage profit we get is exactly the number above.

## How does strike price influence the price of options
We consider the option on a non-dividend paying stock for simplicity. 
For two options with the same expiry date but different strike price $K_1 \leq K_2$, we have 
$$C_t(K_1)\geq C_t(K_2)$$ 
$$P_t(K_1)\leq P_t(K_2)$$
Moreover, the difference between the price of those two options should be less than or equal to the difference of their strike price. we have 
$$C_t(K_1) - C_t(K_2)\leq K_2-K_1$$
$$P_t(K_2) - P_t(K_1)\leq K_2-K_1$$

### Problem 9.2

In [30]:
print("The present value of the divided is given by", call_given_put(P_t = 2.64, S_t = 32, K = 30, r = 0.04, dt = 0.5)-4.29)
32 - 30 * math.exp(-0.04 * 0.5) - 4.29 + 2.64

The present value of the divided is given by 0.9440398007973441


0.9440398007973436

### Problem 9.3

In [31]:
print("the rate of return is", math.log(815/770, math.e))

the rate of return is 0.056797598393133246


b). We see that the rate of return above is greater than the risk-free rate. So we should borrow money from the bank, and purchase the above portfolio. Then we get profits for free.

In [32]:
print("The correct difference between their price should be", call_given_put(P_t = 0, S_t = 800, K = 815, r = 0.05, dt = 1))

The correct difference between their price should be 24.748019031918034


In [33]:
K = 780
print("The correct difference between the price of call and put under the strike price", K,  "should be", 
call_given_put(P_t = 0, S_t = 800, K = K, r = 0.05, dt = 1))
K = 800
print("The correct difference between the price of call and put under the strike price", K,  "should be", 
call_given_put(P_t = 0, S_t = 800, K = K, r = 0.05, dt = 1))
K = 820
print("The correct difference between the price of call and put under the strike price", K,  "should be", 
call_given_put(P_t = 0, S_t = 800, K = K, r = 0.05, dt = 1))
K = 840
print("The correct difference between the price of call and put under the strike price", K,  "should be", 
call_given_put(P_t = 0, S_t = 800, K = K, r = 0.05, dt = 1))

The correct difference between the price of call and put under the strike price 780 should be 58.04104888944312
The correct difference between the price of call and put under the strike price 800 should be 39.01646039942875
The correct difference between the price of call and put under the strike price 820 should be 19.9918719094145
The correct difference between the price of call and put under the strike price 840 should be 0.9672834194002462
